# Extraction of all skills from the database

In [3]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import zipfile
from rdflib import Graph

run only once to generate the graph serialized nt file

In [5]:
# with zipfile.ZipFile("ESCOdataset.rdf.zip", "r") as z:
    # print(z.namelist()) to get the name of the unzipped file (idk why it is different)
    # with z.open("esco-v1.2.0.rdf") as f:
    #     g = Graph()
    #     g.parse(f, format = "application/rdf+xml")

In [7]:
#Anyi0711
g = Graph()
with open("data/esco-v1.2.0.rdf","rb") as f:
    g.parse(f, format="application/rdf+xml")

KeyboardInterrupt: 

In [8]:
g.serialize("esco.nt", format="nt", encoding="utf-8") #auto uses UTF8 encoding

KeyboardInterrupt: 

# Run from here if no need to reread the entire graph from the source file (much faster from here :))

In [9]:
g = Graph()
g.parse("esco.nt", format ="nt")

<Graph identifier=N20b3c63f6d8742b8bdbf1b7f6c1713a2 (<class 'rdflib.graph.Graph'>)>

In [10]:
def query(q):
    return g.query(q)
getTripleCount = "SELECT (COUNT(*) AS ?triples) WHERE {?s ?p ?o.}"

for row in g.query(getTripleCount):

    print("Total triples: ", row.triples)

Total triples:  626293


In [15]:
getSkills = """ 
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX esco: <http://data.europa.eu/esco/model#>
    SELECT ?skill ?label 
    WHERE {
        ?skill a esco:Skill; skos:prefLabel ?label.
        FILTER(lang(?label) = "en")
    }
"""
skills = query(getSkills)
print(skills)
print(type(skills))

skill_list = []
for skill in skills:
    # toPython --> convert Literal to only the text part we want
    skill_list.append(skill.label.toPython())
print(len(skill_list))
print(skill_list[:10])

df_skills = pd.DataFrame({"skill": skill_list})
df_skills.to_csv("skill_df.csv", encoding = "utf8")

<class 'rdflib.plugins.sparql.processor.SPARQLResult'>
186
['interpret performance concepts in the creative process', 'inspect electrical supplies', 'build contacts to maintain news flow', 'display betting information', 'inspect cables', 'interact verbally in Kazakh', 'assess the life cycle of resources', 'prevent money laundering in gambling', 'prepare flower arrangements', 'manage maintenance operations']


In [ ]:
skills = query(getSkills)

# Convert to DataFrame (with URI)
skill_uris  = [str(s.skill) for s in skills]
skill_labels = [s.label.toPython() for s in skills]

df_skills_wuri = pd.DataFrame({"uri": skill_uris, "text": skill_labels})
print(f"number of skills{len(df_skills_wuri):,}")
print(df_skills_wuri.head())

number of skills14,257
                                                 uri  \
0  http://data.europa.eu/esco/skill/f82b05a7-128b...   
1  http://data.europa.eu/esco/skill/045f71e6-0699...   
2  http://data.europa.eu/esco/skill/6e04cc9f-d409...   
3  http://data.europa.eu/esco/skill/d54d7c00-87c4...   
4  http://data.europa.eu/esco/skill/19ba11f3-9c99...   

                                    text  
0  advise on waste management procedures  
1                          advise others  
2                        persuade others  
3                                Walloon  
4                 carry out calculations  


In [ ]:
# Save df_skills_wuri to CSV
df_skills_wuri.to_csv("skills_en_one_per_skill.csv", index=False, encoding="utf-8")
print(f"Saved {len(df_skills_wuri):,} skills to skills_en_one_per_skill.csv")
print(f"Columns: {list(df_skills_wuri.columns)}")


Saved 14,257 skills to skills_en_one_per_skill.csv
Columns: ['uri', 'text']
